# Predictor Matrix Generation {-}
## EE627 {-}
### Connor DePalma & Daniel Pelis - Ground Truth {-}

In [1]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import IntegerType, StructType, StructField, StringType
import pyspark.sql.functions as sf
import pandas as pd

In [2]:
training = spark.read.csv("trainItem.data", header = False)

schema = StructType([
    StructField("UserID", StringType(), True),
    StructField("TrackID", StringType(), True),
    StructField("AlbumID", StringType(), True),
    StructField("ArtistID", StringType(), True),
    StructField("GenreID_1", StringType(), True),
    StructField("GenreID_2", StringType(), True),
    StructField("GenreID_3", StringType(), True),
    StructField("GenreID_4", StringType(), True),
    StructField("GenreID_5", StringType(), True),
    StructField("GenreID_6", StringType(), True),
    StructField("GenreID_7", StringType(), True),
    StructField("GenreID_8", StringType(), True),
    StructField("GenreID_9", StringType(), True),
    StructField("GenreID_10", StringType(), True),
    StructField("GenreID_11", StringType(), True),
    StructField("GenreID_12", StringType(), True),
    StructField("GenreID_13", StringType(), True),
    StructField("GenreID_14", StringType(), True),
    StructField("GenreID_15", StringType(), True),
    StructField("GenreID_16", StringType(), True),
])
testing_hr = spark.read.option("delimiter","|").csv("testTrack_hierarchy.txt", header = False, schema=schema)
training.show(5)

+------+------+---+
|   _c0|   _c1|_c2|
+------+------+---+
|199808|248969| 90|
|199808|  2663| 90|
|199808| 28341| 90|
|199808| 42563| 90|
|199808| 59092| 90|
+------+------+---+
only showing top 5 rows



In [3]:
training = training.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "rating")
training.show(5)

+------+------+------+
|userID|itemID|rating|
+------+------+------+
|199808|248969|    90|
|199808|  2663|    90|
|199808| 28341|    90|
|199808| 42563|    90|
|199808| 59092|    90|
+------+------+------+
only showing top 5 rows



In [4]:
training = training.withColumn("userID", training["userID"].cast(IntegerType()))
training = training.withColumn("itemID", training["itemID"].cast(IntegerType()))
training = training.withColumn("rating", training["rating"].cast('float'))
training.show(3)

+------+------+------+
|userID|itemID|rating|
+------+------+------+
|199808|248969|  90.0|
|199808|  2663|  90.0|
|199808| 28341|  90.0|
+------+------+------+
only showing top 3 rows



In [5]:
testing_hr = testing_hr.withColumn("UserID", testing_hr["UserID"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("TrackID", testing_hr["TrackID"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("AlbumID", testing_hr["AlbumID"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("ArtistID", testing_hr["ArtistID"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("GenreID_1", testing_hr["GenreID_1"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("GenreID_2", testing_hr["GenreID_2"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("GenreID_3", testing_hr["GenreID_3"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("GenreID_4", testing_hr["GenreID_4"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("GenreID_5", testing_hr["GenreID_5"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("GenreID_6", testing_hr["GenreID_6"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("GenreID_7", testing_hr["GenreID_7"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("GenreID_8", testing_hr["GenreID_8"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("GenreID_9", testing_hr["GenreID_9"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("GenreID_10", testing_hr["GenreID_10"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("GenreID_11", testing_hr["GenreID_11"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("GenreID_12", testing_hr["GenreID_12"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("GenreID_13", testing_hr["GenreID_13"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("GenreID_14", testing_hr["GenreID_14"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("GenreID_15", testing_hr["GenreID_15"].cast(IntegerType()))
testing_hr = testing_hr.withColumn("GenreID_16", testing_hr["GenreID_16"].cast(IntegerType()))

In [6]:
testing_hr.show(5)

+------+-------+-------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+
|UserID|TrackID|AlbumID|ArtistID|GenreID_1|GenreID_2|GenreID_3|GenreID_4|GenreID_5|GenreID_6|GenreID_7|GenreID_8|GenreID_9|GenreID_10|GenreID_11|GenreID_12|GenreID_13|GenreID_14|GenreID_15|GenreID_16|
+------+-------+-------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+
|199810| 208019| 209288|    null|     null|     null|     null|     null|     null|     null|     null|     null|     null|      null|      null|      null|      null|      null|      null|      null|
|199810|  74139| 277282|  271146|   113360|   173467|   173655|   192976|   146792|    48505|   133159|     null|     null|      null|      null|      null|      null|      null|      null|      n

In [7]:
users = sorted([i.userID for i in testing_hr.select('userID').distinct().collect()])

In [8]:
len(users)

20000

In [9]:
users[:10]

[199810,
 199812,
 199813,
 199814,
 199815,
 199816,
 199817,
 199818,
 199819,
 199820]

### Change the slice here to grab different consecutive sections of the data. Memory constraints required us to generate the testing csv by running this notebook 4 times. 

In [10]:
users = users[15000:20000]

In [11]:
users[:10]

[236600,
 236601,
 236605,
 236610,
 236621,
 236622,
 236626,
 236627,
 236631,
 236632]

In [12]:
users[-1]

249010

In [13]:
testing_hr_bet = testing_hr.toPandas()

In [14]:
training_bet = training.filter(training.userID.isin(users)).toPandas()

In [15]:
testing_hr_bet

,UserID,TrackID,AlbumID,ArtistID,GenreID_1,GenreID_2,GenreID_3,GenreID_4,GenreID_5,GenreID_6,GenreID_7,GenreID_8,GenreID_9,GenreID_10,GenreID_11,GenreID_12,GenreID_13,GenreID_14,GenreID_15,GenreID_16
0,199810,208019,209288.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,199810,74139,277282.0,271146.0,113360.0,173467.0,173655.0,192976.0,146792.0,48505.0,133159.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,199810,9903,NaN,NaN,33722.0,123396.0,79926.0,73523.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,199810,242681,190640.0,244574.0,61215.0,17453.0,274088.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,199810,18515,146344.0,33168.0,19913.0,48505.0,154024.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,249010,72192,258175.0,258848.0,61215.0,17453.0,274088.0,88853.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119996,249010,86104,94638.0,252184.0,33204.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119997,249010,186634,257363.0,190627.0,173467.0,48505.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119998,249010,293818,289167.0,189054.0,189467.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
training_bet

,userID,itemID,rating
0,236600,116812,0.0
1,236600,122616,90.0
2,236600,153565,30.0
3,236600,103870,30.0
4,236600,143789,70.0
...,...,...,...
2742023,249010,118658,90.0
2742024,249010,225510,90.0
2742025,249010,193079,90.0
2742026,249010,61874,90.0


In [17]:
predictor = []
for user in users:
    user_train = training_bet.loc[training_bet["userID"]==user]
    user_test = testing_hr_bet.loc[testing_hr_bet["UserID"]==user].drop(columns=["UserID","TrackID"])
    itemID = list(user_train.itemID.values)
    rating = list(user_train.rating.values)
    for idx, item in enumerate(itemID):
        user_test.replace(to_replace=item,value=rating[idx],inplace=True)
    predictor.append(user_test)
predictor = pd.concat(predictor, axis=0).reset_index(drop=True)

In [18]:
predictor[predictor>100] = None

In [19]:
predictor

,AlbumID,ArtistID,GenreID_1,GenreID_2,GenreID_3,GenreID_4,GenreID_5,GenreID_6,GenreID_7,GenreID_8,GenreID_9,GenreID_10,GenreID_11,GenreID_12,GenreID_13,GenreID_14,GenreID_15,GenreID_16
0,NaN,NaN,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90.0,90.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90.0,90.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,80.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29997,90.0,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
numGenre = predictor.iloc[:,2:].count(axis=1)
avgGenre = predictor.iloc[:,2:].mean(axis=1)
maxGenre = predictor.iloc[:,2:].max(axis=1)
minGenre = predictor.iloc[:,2:].min(axis=1)
varGenre = predictor.iloc[:,2:].var(axis=1)
medGenre = predictor.iloc[:,2:].median(axis=1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


In [21]:
genreStat = pd.DataFrame()
genreStat['num_genre'] = numGenre
genreStat['max'] = maxGenre
genreStat['min'] = minGenre
genreStat['mean'] = avgGenre
genreStat['variance'] = varGenre
genreStat['median'] = medGenre

In [22]:
genreStat

,num_genre,max,min,mean,variance,median
0,1,80.0,80.0,80.0,NaN,80.0
1,1,80.0,80.0,80.0,NaN,80.0
2,1,80.0,80.0,80.0,NaN,80.0
3,1,80.0,80.0,80.0,NaN,80.0
4,2,80.0,80.0,80.0,0.0,80.0
...,...,...,...,...,...,...
29995,0,NaN,NaN,NaN,NaN,NaN
29996,0,NaN,NaN,NaN,NaN,NaN
29997,0,NaN,NaN,NaN,NaN,NaN
29998,0,NaN,NaN,NaN,NaN,NaN


In [23]:
testing_hr_bet[testing_hr_bet.UserID.isin(users)].UserID.reset_index(drop=True)

0        236600
1        236600
2        236600
3        236600
4        236600
          ...  
29995    249010
29996    249010
29997    249010
29998    249010
29999    249010
Name: UserID, Length: 30000, dtype: int32

In [24]:
predictor

,AlbumID,ArtistID,GenreID_1,GenreID_2,GenreID_3,GenreID_4,GenreID_5,GenreID_6,GenreID_7,GenreID_8,GenreID_9,GenreID_10,GenreID_11,GenreID_12,GenreID_13,GenreID_14,GenreID_15,GenreID_16
0,NaN,NaN,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90.0,90.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90.0,90.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,80.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29997,90.0,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
pred = pd.DataFrame()
pred['userID'] = testing_hr_bet[testing_hr_bet.UserID.isin(users)].UserID.reset_index(drop=True)
pred['trackID'] = testing_hr_bet[testing_hr_bet.UserID.isin(users)].TrackID.reset_index(drop=True)
pred['recommendation'] = 0
pred['album'] = predictor.AlbumID
pred['artist'] = predictor.ArtistID
pred = pd.concat([pred, genreStat], axis=1)

In [26]:
testing_hr_bet[testing_hr_bet.UserID.isin(users)].UserID

90000     236600
90001     236600
90002     236600
90003     236600
90004     236600
           ...  
119995    249010
119996    249010
119997    249010
119998    249010
119999    249010
Name: UserID, Length: 30000, dtype: int32

In [27]:
pred

,userID,trackID,recommendation,album,artist,num_genre,max,min,mean,variance,median
0,236600,173879,0,NaN,NaN,1,80.0,80.0,80.0,NaN,80.0
1,236600,207784,0,NaN,NaN,1,80.0,80.0,80.0,NaN,80.0
2,236600,91145,0,90.0,90.0,1,80.0,80.0,80.0,NaN,80.0
3,236600,160650,0,90.0,90.0,1,80.0,80.0,80.0,NaN,80.0
4,236600,183711,0,NaN,NaN,2,80.0,80.0,80.0,0.0,80.0
...,...,...,...,...,...,...,...,...,...,...,...
29995,249010,72192,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
29996,249010,86104,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
29997,249010,186634,0,90.0,90.0,0,NaN,NaN,NaN,NaN,NaN
29998,249010,293818,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [28]:
# values = {'A': 0, 'B': 1, 'C': 2, 'D': 3}

pred = pred.fillna(0)

In [29]:
pred

,userID,trackID,recommendation,album,artist,num_genre,max,min,mean,variance,median
0,236600,173879,0,0.0,0.0,1,80.0,80.0,80.0,0.0,80.0
1,236600,207784,0,0.0,0.0,1,80.0,80.0,80.0,0.0,80.0
2,236600,91145,0,90.0,90.0,1,80.0,80.0,80.0,0.0,80.0
3,236600,160650,0,90.0,90.0,1,80.0,80.0,80.0,0.0,80.0
4,236600,183711,0,0.0,0.0,2,80.0,80.0,80.0,0.0,80.0
...,...,...,...,...,...,...,...,...,...,...,...
29995,249010,72192,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
29996,249010,86104,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
29997,249010,186634,0,90.0,90.0,0,0.0,0.0,0.0,0.0,0.0
29998,249010,293818,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0


### Need to append to file since this will need to be run multiple times in order to collect all of the test samples (due to memory constraints) 

In [30]:
#pred.to_csv("./test_set.csv",index=False)
pred.to_csv('./test_set.csv', mode='a', header=False, index=False)